In [1]:
import pandas as pd
#import graphlab as gb
import matplotlib.pyplot as plt
import numpy as np
#import seaborn as sb
get_ipython().magic(u'matplotlib inline')
from sklearn import metrics
from utility import *
from datacleaner import autoclean
from sklearn.preprocessing import OneHotEncoder
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.svm import SVC,LinearSVC
from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier,ExtraTreeClassifier,export_graphviz
from sklearn.linear_model import LogisticRegression,SGDClassifier,RidgeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.linear_model.perceptron import Perceptron
from sklearn.feature_selection import VarianceThreshold
from sklearn.cross_validation import train_test_split

In [30]:
#def strip_term(data):
#,'addr_state'
tdf=pd.read_csv('loan_cleaned.csv')[['loan_amnt','term','int_rate','grade','sub_grade','emp_length','home_ownership',
          'annual_inc','verification_status','purpose','dti','pub_rec',
          'acc_now_delinq','delinq_2yrs','mths_since_last_delinq','open_acc','total_acc',
          'labels','addr_state','zip_code']]

In [3]:
tdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254190 entries, 0 to 254189
Data columns (total 18 columns):
loan_amnt                 254190 non-null float64
term                      254190 non-null object
int_rate                  254190 non-null float64
grade                     254190 non-null object
sub_grade                 254190 non-null object
emp_length                254190 non-null int64
home_ownership            254190 non-null object
annual_inc                254190 non-null float64
verification_status       254190 non-null object
purpose                   254190 non-null object
dti                       254190 non-null float64
pub_rec                   254190 non-null float64
acc_now_delinq            254190 non-null float64
delinq_2yrs               254190 non-null float64
mths_since_last_delinq    254190 non-null float64
open_acc                  254190 non-null float64
total_acc                 254190 non-null float64
labels                    254190 non-null int64

In [4]:
"""categorical=['term','grade','sub_grade','home_ownership','purpose']#,'addr_state']
tdf=encode_onehot(tdf,categorical)
"""

"categorical=['term','grade','sub_grade','home_ownership','purpose']#,'addr_state']\ntdf=encode_onehot(tdf,categorical)\n"

In [5]:
df=autoclean(input_dataframe=tdf,drop_nans=True)
df.head()

,loan_amnt,term,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,pub_rec,acc_now_delinq,delinq_2yrs,mths_since_last_delinq,open_acc,total_acc,labels
0,5000.0,0,10.65,1,6,10,5,24000.0,2,1,27.65,0.0,0.0,0.0,0.0,3.0,9.0,1
1,2500.0,1,15.27,2,13,1,5,30000.0,1,0,1.00,0.0,0.0,0.0,0.0,3.0,4.0,0
2,2400.0,0,15.96,2,14,10,5,12252.0,0,11,8.72,0.0,0.0,0.0,0.0,2.0,10.0,1
3,10000.0,0,13.49,2,10,10,5,49200.0,1,9,20.00,0.0,0.0,0.0,35.0,10.0,37.0,1
4,5000.0,0,7.90,0,3,3,5,36000.0,1,13,11.20,0.0,0.0,0.0,0.0,9.0,12.0,1


In [6]:
#tdf=pd.read_csv('loan_cleaned.csv')[feature_cols]
#print tdf.term.unique()
#print df.term.unique()
#tdf.info()

In [7]:
print sum(df['labels']==1)
print sum(df['labels']==0)
print len(df)

207723
46467
254190


In [8]:
df_balanced=balance_data(df,'labels',0,1)
#df_balanced=df

In [9]:
print sum(df_balanced['labels']==1)
print sum(df_balanced['labels']==0)
print len(df_balanced)

46467
46467
92934


In [10]:
#df_balanced.columns

In [11]:
#full meaningful features
"""features=['loan_amnt','funded_amnt_inv','term','int_rate','installment','grade','sub_grade',
      'emp_title','emp_length','home_ownership','annual_inc',
      'verification_status','issue_d','purpose','title','addr_state','dti',
      'delinq_2yrs','earliest_cr_line','inq_last_6mths',
      'mths_since_last_delinq','open_acc','pub_rec','revol_bal',
      'revol_util','total_acc','initial_list_status','out_prncp',
      'out_prncp_inv','total_pymnt','total_pymnt_inv',
      'total_rec_prncp','total_rec_int','total_rec_late_fee',
      'recoveries','collection_recovery_fee','last_pymnt_d',
      'last_pymnt_amnt','last_credit_pull_d',
      'collections_12_mths_ex_med','application_type',
      'acc_now_delinq','tot_coll_amt','tot_cur_bal','total_rev_hi_lim']"""

"features=['loan_amnt','funded_amnt_inv','term','int_rate','installment','grade','sub_grade',\n      'emp_title','emp_length','home_ownership','annual_inc',\n      'verification_status','issue_d','purpose','title','addr_state','dti',\n      'delinq_2yrs','earliest_cr_line','inq_last_6mths',\n      'mths_since_last_delinq','open_acc','pub_rec','revol_bal',\n      'revol_util','total_acc','initial_list_status','out_prncp',\n      'out_prncp_inv','total_pymnt','total_pymnt_inv',\n      'total_rec_prncp','total_rec_int','total_rec_late_fee',\n      'recoveries','collection_recovery_fee','last_pymnt_d',\n      'last_pymnt_amnt','last_credit_pull_d',\n      'collections_12_mths_ex_med','application_type',\n      'acc_now_delinq','tot_coll_amt','tot_cur_bal','total_rev_hi_lim']"

In [12]:
#selected_features.head()

In [13]:
#features available at loan application time
"""feature_cols=['loan_amnt','term','int_rate','grade','sub_grade','emp_length','home_ownership',
          'annual_inc','verification_status','purpose','addr_state','dti','pub_rec',
          'acc_now_delinq','delinq_2yrs','mths_since_last_delinq','open_acc','total_acc']#earliest_cr_line"""
feature_cols=list(df_balanced.columns)
feature_cols.remove('labels')
target=df_balanced['labels']

In [14]:
"""feature_selector=VarianceThreshold(threshold=0.2)
selected_features=feature_selector.fit_transform(df_balanced[features])
print len(features)
print len(selected_features[0])"""

'feature_selector=VarianceThreshold(threshold=0.2)\nselected_features=feature_selector.fit_transform(df_balanced[features])\nprint len(features)\nprint len(selected_features[0])'

In [15]:
selected_features=df_balanced[feature_cols]

In [16]:
selected_features.head()

,loan_amnt,term,int_rate,grade,sub_grade,emp_length,home_ownership,annual_inc,verification_status,purpose,dti,pub_rec,acc_now_delinq,delinq_2yrs,mths_since_last_delinq,open_acc,total_acc
48456,12000.0,0,17.76,3,15,1,1,67000.0,1,2,11.96,0.0,0.0,0.0,0.0,12.0,17.0
68125,17450.0,1,10.64,1,6,10,1,65000.0,2,1,18.27,0.0,0.0,0.0,0.0,12.0,19.0
82591,11200.0,0,12.12,1,7,1,4,45000.0,1,2,14.72,0.0,0.0,0.0,37.0,12.0,27.0
216845,4500.0,0,14.64,2,12,10,5,68000.0,2,2,25.80,0.0,0.0,1.0,4.0,10.0,33.0
108,12000.0,0,11.71,1,7,2,1,126000.0,0,2,13.32,0.0,0.0,0.0,74.0,17.0,38.0


In [17]:
x_train, x_test, y_train, y_test = train_test_split(selected_features,target, test_size=1/4.0, random_state=4082,stratify=target)

In [18]:
print sum(y_train==1)
print sum(y_train==0)

print sum(y_test==1)
print sum(y_test==0)

34850
34850
11617
11617


In [27]:
#clf=DecisionTreeClassifier()
#clf=ExtraTreeClassifier()
#clf=LogisticRegression()
#clf=RidgeClassifier()
#clf=XGBClassifier(learning_rate =0.1, n_estimators=1000, max_depth=10, min_child_weight=1, gamma=0, subsample=0.8,
#                  colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
#clf= XGBClassifier()
#clf=Perceptron(n_iter=500)
#clf = AdaBoostClassifier(n_estimators=150)
#clf=GradientBoostingClassifier(n_estimators=150)
clf = RandomForestClassifier(n_estimators=150)
#clf = LinearSVC(max_iter=100,random_state=9701)

In [28]:
clf.fit(x_train,y_train)
test_pred=clf.predict(x_test)
train_pred=clf.predict(x_train)

In [29]:
print "#miss-classification (train): ",sum(train_pred!=y_train),"/",len(train_pred)
print "#miss-classification (test): ",sum(test_pred!=y_test),"/",len(test_pred)
print "train accuracy: ",100.0*sum(train_pred==y_train)/float(len(train_pred)),"%"
print "test accuracy: ",100.0*sum(test_pred==y_test)/float(len(test_pred)),"%"

#miss-classification (train):  0 / 69700
#miss-classification (test):  8250 / 23234
train accuracy:  100.0 %
test accuracy:  64.4916932082 %


In [22]:
#export_graphviz(clf,out_file='tree.dot')  

In [23]:
#!dot -Tpng tree.dot -o tree.png